# Binary Text Classification scenario with RAI Dashboard

This notebook demonstrates the use of the `responsibleai` API to binary text classification scenario end to end where a [Huggingface model](https://huggingface.co/docs/transformers/tasks/sequence_classification) will be trained on Fabricated Financial News dataset. The model predicts **the category a Financial News article will fall under.** There are 7 Categories of the Financial news:

1. Banking and finance - Debt Market
2. Business
3. Cryptocurrency
4. Financial Regulations
5. Personal Finance
6. Real Estate
7. Stock Market Updates

The Data Dictionary can be accessed through the following link: [Data_dictionary_Finance](link-URL)

The Notebook walks through the API calls necessary to create a widget with model analysis insights, then guides a visual analysis of the model. The Notebook uses [Responsibleai_text Toolbox](https://github.com/microsoft/responsible-ai-toolbox/tree/main/responsibleai_text) to generate the dashboard.



## **Installation**  

If you are **running the notebook for the first time**, you need to follow a few of steps for smooth execution of notebook:

1. Un-comment the 2 cells below.
2. Run the 2 cells.
3. After execution of these cells, comment the cells.
4. Re-start the kernel
5. Continue with running of all cells.


**Reminder** -- Be sure to set your kernel to "Python 3.10 - SDK v2," via the drop-down menu at the right end of the taskbar. 

### Install Required dependencies

**Make sure it comment the below cell while executing the notebook more than once**

In [ ]:
%pip install azure-ai-ml
%pip install raiutils
%pip install azureml-rai-utils

%pip install datasets
%pip install "pandas<2.0.0"
%pip install scikit-learn
%pip install mltable
%pip install azure.ai.ml
%pip install azure.identity
%pip install mltable
%pip install transformers
%pip install torch
%pip install openpyxl

First, it is imperative to define the versions of the Responsible AI (RAI) components accessible within the workspace. These specifications were explicitly indicated during the upload of the components.

In [ ]:
version_string = "0.0.13"

Furthermore, it is essential to provide the designation of the compute cluster desired for utilization in AzureML. Subsequently, in this notebook, we will generate the cluster if it does not currently exist.

In [ ]:
compute_name = "raitextcluster"

Lastly, we must stipulate a version for the data and components that will be generated during the execution of this notebook. This version should be exclusive to the workspace, and its actual value holds no significance, as long as it is unique.

In [ ]:
rai_example_version_string = "13"

In [ ]:
import os
import json

# import datasets
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import zipfile
from io import BytesIO
import requests

## Accessing the Data

The following section examines the code necessary to read datasets and a model using components in AzureML.

**Note:** It is advisable to keep the data file and the notebook in same folder. In case they are kept in separate folders, update the path to the dataset  

In [ ]:
# Load the labeled dataset from Excel

# Download the blob data from the provided URL
data_location = "https://publictestdatasets.blob.core.windows.net/data/RAI_fabricated_text_classification_data.zip"
response = requests.get(data_location)
blob_content = response.content

with zipfile.ZipFile(BytesIO(blob_content), "r") as zip_ref:
    file_list = zip_ref.namelist()
    if len(file_list) > 0:
        # Assume the first file in the zip contains the data
        inner_blob_name = file_list[0]
        inner_blob_content = zip_ref.read(inner_blob_name)
        df = pd.read_excel(BytesIO(inner_blob_content))

# df = pd.read_excel("./Text_classification_dataset.xlsx")
df = df[["Article Description", "Category"]]
df.columns = ["text", "label"]
df

Validating the the category names and the number of categories

In [ ]:
list(df["label"].unique())

### Splitting the Data into train and test datasets

In [ ]:
train_data, test_data = train_test_split(
    df, test_size=0.20, random_state=0, stratify=df["label"]
)
test_data

### Saving test & train files
With the data now split into 'train' and 'test' DataFrames, we save them as parquet files in preparation for upload into AzureML.

In [ ]:
train_pq_filename = "Financial_news_train_data.parquet"
train_data_folder = "./data_news_classification/train/"
train_data_path = train_data_folder + train_pq_filename

os.makedirs(train_data_folder, exist_ok=True)
train_data.to_parquet(train_data_path, index=False)
train_df = pd.read_parquet(train_data_path)
train_df

In [ ]:
test_pq_filename = "Financial_news_test_data.parquet"
test_data_folder = "./data_news_classification/test/"
test_data_path = test_data_folder + test_pq_filename

os.makedirs(test_data_folder, exist_ok=True)
test_data.to_parquet(test_data_path, index=False)
test_df = pd.read_parquet(test_data_path)
test_df

## Get the Data to AzureML


We are going to create two Data assets in AzureML, one for the train and another for the test. The first step is to create an `MLClient` to perform interactions with AzureML:

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# Handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace,
)
print(ml_client)

### Create a data asset (URI file) to register the Data into workspace
This is essential,  as the dashboard recognizes only registered assets. 

Reference:
https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK

**Note:** Change the asset name of the below file if the train/test data has changed

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_train_data = "Financial_News_train_URI_file"

try:
    # Try getting data already registered in workspace
    train_data = ml_client.data.get(
        name=input_train_data,
        version=rai_example_version_string,
    )

except Exception as e:
    train_data = Data(
        path=train_data_path,
        type=AssetTypes.URI_FILE,
        description="RAI News article train data URI File",
        name=input_train_data,
        version=rai_example_version_string,
    )
    ml_client.data.create_or_update(train_data)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

input_test_data = "Financial_News_test_URI_file"

try:
    # Try getting data already registered in workspace
    test_data = ml_client.data.get(
        name=input_test_data,
        version=rai_example_version_string,
    )

except Exception as e:
    test_data = Data(
        path=test_data_path,
        type=AssetTypes.URI_FILE,
        description="RAI News article test data URI File",
        name=input_test_data,
        version=rai_example_version_string,
    )
    ml_client.data.create_or_update(test_data)

### Creating the training script RAI Dashboard

## A model training pipeline

To simplify the model creation process, we're going to use a pipeline. This will have two stages:

1. The actual training component
2. A model registration component

We have to register the model in AzureML in order for our RAI text insights components to use it. In this notebook we will be training and registering the model in a single pipeline



We start by creating a directory to hold the component source:

In [ ]:
import os

os.makedirs("Text_classification_component_src", exist_ok=True)

### The Training Component

This Python notebook contains code for a text classification pipeline that uses the BERT model for fine-tuning and predicting the labels of financial news articles. The pipeline is implemented as a Python script and will be run to create a model, fine-tune it, and register it in Azure ML.

#### Pipeline Steps

1. **Data Loading**: The pipeline starts by loading the financial news dataset from a parquet file.

2. **Data Preprocessing**: The text column containing the news articles' text is used for training. The target column name (containing the labels) is specified as an argument.

3. **Tokenization**: The text data is tokenized using the [BERT tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer). The input texts are converted into input encodings and attention masks to feed into the BERT model.

4. **Model Fine-Tuning**: The [BERT model](https://huggingface.co/bert-base-uncased) is loaded, and its last layer is fine-tuned using the training data. The number of epochs and learning rate are configurable.

5. **Prediction Pipeline**: After fine-tuning, a prediction pipeline is built using the transformers pipeline object. This pipeline will allow us to make predictions on new text data. It also saves the trained model to a specified output path using MLFlow. 

6. **Model Registration**: The trained model is registered in Azure ML with a specific model name and a suffix based on the current timestamp.

7. **JSON Output**: A JSON file is written, containing information about the registered model, such as its ID and version, and saved in a specified directory.


### Huggingface Wrapper for Text Classification Model

The `HuggingfaceWrapper` class is a Python model wrapper designed to work with the [Hugging Face](https://huggingface.co/) library. It enables easy integration of Hugging Face's text classification models into Azure ML pipelines using the `mlflow.pyfunc.PythonModel` base class.


#### Purpose

**This Wrapper helps wrapping the model in *Pyfunc* flavour while it is logged via MLFLow.**

The purpose of this wrapper class is to provide a convenient interface to make predictions and inference using Hugging Face's text classification pipelines. It encapsulates the functionality of SHAP (SHapley Additive exPlanations) and ensures that SHAP is installed before using it for explanation purposes.

#### Class Methods

**`__init__(self, pipeline)`**

The constructor initializes the `HuggingfaceWrapper` object with a text classification pipeline provided as an argument. If SHAP is not installed, it raises an `ImportError` and informs the user to install it to use SHAP for explanations.

**`predict(self, context, model_input=None)`**

This method is used to make predictions using the wrapped text classification model. It takes the context and an optional `model_input` argument. If `model_input` is not provided, it uses the `context` parameter for inference. The method returns an array containing the indices of the highest probability predictions for each input.

**`predict_proba(self, dataset)`**

The `predict_proba` method returns the predictions probabilities (scores) for a given dataset using the wrapped model.

#### Dependencies

This script requires the following libraries and packages to be installed:

- azureml-core
- torch
- transformers
- mlflow
- pandas
- numpy
- sklearn

Make sure to have these packages installed before running the pipeline script.

Let's proceed with running the pipeline and registering the model in Azure ML.

In [ ]:
%%writefile Text_classification_component_src/training_script.py

import argparse
import os, pathlib
import shutil
import tempfile
import time
import pandas as pd
import pickle
import json
import numpy as np

from azureml.core import Run

import torch
from sklearn.preprocessing import LabelEncoder

import mlflow
import mlflow.pyfunc
from transformers import pipeline, AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from transformers.models.auto import AutoConfig

from ml_wrappers import wrap_model
from ml_wrappers.common.warnings_suppressor import shap_warnings_suppressor
from ml_wrappers.common.constants import ModelTask


with shap_warnings_suppressor():
    try:
        from shap import models
        shap_installed = True
    except BaseException:
        shap_installed = False

text_column = "text"

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", type=str, help="Path to training data")
    parser.add_argument("--target_column_name", type=str, help="Name of target column")
    parser.add_argument("--model_info_output_path", type=str, help="Path to write model info JSON")
    parser.add_argument("--model_base_name", type=str, help="Name of the registered model")
    parser.add_argument("--model_name_suffix", type=int, help="Set negative to use epoch_secs")
    # parse args
    args = parser.parse_args()

    # return args
    return args

class HuggingfaceWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, pipeline):
        self._model = self

        if not shap_installed:
            raise ImportError("SHAP is not installed. Please install it " +
                              "to use WrappedTextClassificationModel.")
        self._wrapped_model = models.TransformersPipeline(pipeline)

    def predict(self, context, model_input=None):
        if model_input is None:
            model_input = context # resolve positional inputs if only one arg passed
        
        # get all the probabilities
        scores = self.predict_proba(model_input)

        # find index of highest probability for each prediction
        scores_list = scores.tolist()        
        indices =[pred.index(max(pred)) for pred in scores_list]       
                
        return np.array(indices)

    def predict_proba(self, dataset):
        return self._wrapped_model(dataset)

def main(args):
    current_experiment = Run.get_context().experiment
    tracking_uri = current_experiment.workspace.get_mlflow_tracking_uri()
    print("tracking_uri: {0}".format(tracking_uri))
    mlflow.set_tracking_uri(tracking_uri)
    mlflow.set_experiment(current_experiment.name)

    model_name = 'bert-base-uncased'
    text_column_name = 'text'

    # Read in data
    print("Loading financial news dataset")
    df = pd.read_parquet(args.training_data)
    print(df.head(5))

    # Preparing train Data
    label_encoder = LabelEncoder()
    df['encoded_label'] = label_encoder.fit_transform(df[args.target_column_name])
    train_df= df[[text_column_name , 'encoded_label']]

    # Generating encodings
    label2id_encodings = {l: i for (i, l) in enumerate(label_encoder.fit(df[args.target_column_name]).classes_)}
    id2label_encodings = {v: k for k, v in label2id_encodings.items()}

    # Fine Tuning
    print("Loading the model and tokenizer")
    ## Step 1: Load the pre-trained model and tokenizer
    model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(train_df['encoded_label'].unique()),
        id2label=id2label_encodings,
        label2id=label2id_encodings,
        )
    tokenizer = BertTokenizer.from_pretrained(
        model_name,
        )
    
    print("Prepare the text data")
    ## Step 2: Prepare your data
    train_texts = train_df[text_column_name].tolist() # List of training texts
    train_labels = train_df['encoded_label'].tolist()  # List of corresponding training labels

    ### Tokenize the input texts
    train_encodings = tokenizer(train_texts, truncation=True, padding=True)

    ### Convert input encodings and labels to tensors
    train_input_ids = torch.tensor(train_encodings['input_ids'])
    train_attention_masks = torch.tensor(train_encodings['attention_mask'])
    train_labels = torch.tensor(train_labels)

    ## Step 3: Fine-tuning last layer BERT
    ### Define training parameters
    num_epochs = 15
    learning_rate = 3e-5
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    print("Fine-tuning the model")
    model.train()
    for epoch in range(num_epochs):  # Set the desired number of training epochs
        outputs = model(train_input_ids, attention_mask=train_attention_masks, labels=train_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    ### Switch to Evaluation mode    
    _ = model.eval()

    # build a pipeline object to do predictions
    print("Buildling the prediction pipeline")
    # Load the trained model using pipeline
    pred = pipeline(
        'text-classification',
        model=model,
        tokenizer=tokenizer,
        top_k = None,
        device=0 if torch.cuda.is_available() else -1,
        return_all_scores=False # added
    )


    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix
    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    print(f"Registering model as {registered_name}")

    # my_mlflow = PyfuncModel(pred)
    my_mlflow = HuggingfaceWrapper(pred)
    
    if args.model_name_suffix < 0:
        suffix = int(time.time())
    else:
        suffix = args.model_name_suffix

    registered_name = "{0}_{1}".format(args.model_base_name, suffix)
    print(f"Registering model as {registered_name}")

    # Saving model with mlflow
    print("Saving with mlflow")
    mlflow.pyfunc.log_model(
        python_model=my_mlflow,
        registered_model_name=registered_name,
        artifact_path=registered_name,
        )

    print("Writing JSON")
    dict = {"id": "{0}:1".format(registered_name)}
    output_path = os.path.join(args.model_info_output_path, "model_info.json")
    with open(output_path, "w") as of:
        json.dump(dict, fp=of)


# run script
if __name__ == "__main__":
    # add space in logs
    print("*" * 60)
    print("\n\n")

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

### Creating an Azure ML Training Component 

This code cell demonstrates the creation of an Azure Machine Learning (Azure ML) component for training a model on financial news data. The component is defined using a YAML configuration file and is later loaded into the Azure ML workspace for use in the ML pipeline.

#### Component Configuration

The configuration of the component is stored in the YAML format. It specifies the input and output data types, the Python script's location, and the required environment for execution. The inputs to the component include:

- `training_data`: The path to the training data file in parquet format.
- `target_column_name`: The name of the target column in the training data that contains the labels.
- `model_base_name`: The base name to be used for the registered model.
- `model_name_suffix`: An optional integer suffix for the model name. Set to a negative value to use the current timestamp as the suffix.

The output of the component is:

- `model_info_output_path`: The path to the directory where the model info JSON file will be saved.

#### Command Execution

The YAML configuration specifies a Python script, `training_script.py`, that will be executed with the provided inputs and outputs. The script takes the input arguments, performs the model training, and registers the trained model in the Azure ML workspace.

#### Environment

The component specifies the environment required for execution. It uses the environment named `responsibleai-text-ubuntu20.04-py38-cpu` from the Azure ML environment registry.

Let's proceed with creating the Azure ML component and using it in our ML pipeline.


**Executing the below cell multiple times require change in names as the training scripts are not editable**

In [ ]:
from azure.ai.ml import load_component

yaml_contents = (
    f"""
$schema: http://azureml/sdk-2-0/CommandComponent.json
name: rai_financial_news_training_component
display_name: Financial News training component pipeline
version: {rai_example_version_string}
type: command
inputs:
  training_data:
    type: path
  target_column_name:
    type: string
  model_base_name:
    type: string
  model_name_suffix: # Set negative to use epoch_secs
    type: integer
    default: -1
outputs:
  model_info_output_path:
    type: path
code: ./Text_classification_component_src/
environment: azureml://registries/azureml/environments/responsibleai-text-ubuntu20.04-py38-cpu/versions/39
"""
    + r"""
command: >-
  python training_script.py
  --training_data ${{{{inputs.training_data}}}}
  --target_column_name ${{{{inputs.target_column_name}}}}
  --model_base_name ${{{{inputs.model_base_name}}}}
  --model_name_suffix ${{{{inputs.model_name_suffix}}}}
  --model_info_output_path ${{{{outputs.model_info_output_path}}}}
"""
)

yaml_filename = "FinacialNewsTrainingComp.yaml"

with open(yaml_filename, "w") as f:
    f.write(yaml_contents.format(yaml_contents))

train_model_component = load_component(source=yaml_filename)

We need a compute target on which to run our jobs. The following checks whether the compute specified above is present; if not, then the compute target is created.

In [ ]:
from azure.ai.ml.entities import AmlCompute

all_compute_names = [x.name for x in ml_client.compute.list()]

if compute_name in all_compute_names:
    print(f"Found existing compute: {compute_name}")
else:
    my_compute = AmlCompute(
        name=compute_name,
        size="STANDARD_DS4_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=3600,
    )
    ml_client.compute.begin_create_or_update(my_compute)
    print("Initiated compute creation")

## Running a training pipeline

Now that we have our training component, we can run it. We begin by generating a unique name for the mode;

In [ ]:
import time

model_base_name = "Financial_News_classifier"
model_name_suffix = "12486"
device = -1

In [ ]:
from azure.ai.ml import dsl, Input

target_column_name = "label"
encoded_classes = json.dumps(list(df["label"].unique()))

News_train_pq = Input(
    type="uri_file",
    path=f"azureml:{input_train_data}:{rai_example_version_string}",
    mode="download",
)

Next, we define our training pipeline. This has two components. The first is the training component which we defined above. The second is a component to register the model in AzureML:

In [ ]:
@dsl.pipeline(
    compute=compute_name,
    description="RAI computation on Financial News Classification",
    experiment_name=f"RAI_Financial_News_classification_Computation_{model_name_suffix}",
)
def my_training_pipeline(
    target_column_name, training_data, model_base_name, model_name_suffix
):
    trained_model = train_model_component(
        target_column_name=target_column_name,
        training_data=training_data,
        model_base_name=model_base_name,
        model_name_suffix=model_name_suffix,
    )
    trained_model.set_limits(timeout=1200)

    return {}


model_registration_pipeline_job = my_training_pipeline(
    target_column_name, News_train_pq, model_base_name, model_name_suffix
)

With the pipeline definition created, we can submit it to AzureML. We define a helper function to do the submission, which waits for the submitted job to complete:

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job

With the training pipeline defined, we can submit it for execution in AzureML. We define a helper function to wait for the job to complete:

In [ ]:
# This is the actual submission
training_job = submit_and_wait(ml_client, model_registration_pipeline_job)

## Creating the RAI Text Insights 

Now that we have our model, we can generate RAI Text insights for it. We will need the `id` of the registered model, which will be as follows:

In [ ]:
expected_model_id = f"{model_base_name}_{model_name_suffix}:1"
azureml_model_id = f"azureml:{expected_model_id}"
print(expected_model_id)
print(azureml_model_id)

Next, we load the RAI components, so that we can construct a pipeline:

In [ ]:
News_test_pq = Input(
    type="uri_file",
    path=f"azureml:{input_test_data}:{rai_example_version_string}",
    mode="download",
)

registry_name = "azureml"
credential = DefaultAzureCredential()

ml_client_registry = MLClient(
    credential=credential,
    subscription_id=ml_client.subscription_id,
    resource_group_name=ml_client.resource_group_name,
    registry_name=registry_name,
)

rai_text_insights_component = ml_client_registry.components.get(
    name="rai_text_insights", version=version_string
)

We can now specify our pipeline. Complex objects (such as lists of column names) have to be converted to JSON strings before being passed to the components.

In [ ]:
import json
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@dsl.pipeline(
    compute=compute_name,
    description="RAI computation for Financial News Classification",
    experiment_name=f"RAI_Financial_News_classification_Computation_{model_name_suffix}",
)
def rai_text_classification_pipeline(
    target_column_name,
    train_data,
    test_data,
    classes,
    use_model_dependency,
):
    # Initiate the RAIInsights
    rai_text_job = rai_text_insights_component(
        task_type="text_classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        test_dataset=test_data,
        target_column_name=target_column_name,
        classes=classes,
        use_model_dependency=use_model_dependency,
    )
    rai_text_job.set_limits(timeout=6000)

    rai_text_job.outputs.dashboard.mode = "upload"
    rai_text_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_text_job.outputs.dashboard,
        "ux_json": rai_text_job.outputs.ux_json,
    }

In [ ]:
encoded_classes = json.dumps(list(df["label"].unique()))

In [ ]:
encoded_classes

In [ ]:
target_column_name = "label"

Next, we define the pipeline object itself, and ensure that the outputs will be available for download:

In [ ]:
import uuid
from azure.ai.ml import Output


insights_pipeline_job = rai_text_classification_pipeline(
    target_column_name=target_column_name,
    test_data=News_test_pq,
    classes=encoded_classes,
    use_model_dependency=True,
)

rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)
insights_pipeline_job.outputs.ux_json = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/ux_json/",
    mode="upload",
    type="uri_folder",
)

And submit the pipeline to AzureML for execution:

In [ ]:
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

The dashboard should appear in the AzureML portal in the registered model view. The following cell computes the expected URI:

In [ ]:
sub_id = ml_client._operation_scope.subscription_id
rg_name = ml_client._operation_scope.resource_group_name
ws_name = ml_client.workspace_name

expected_uri = f"https://ml.azure.com/model/{expected_model_id}/model_analysis?wsid=/subscriptions/{sub_id}/resourcegroups/{rg_name}/workspaces/{ws_name}"

print(f"Please visit {expected_uri} to see your analysis")

Once this is complete, we can go to the Registered Models view in the AzureML portal, and find the model we have just registered. On the 'Model Details' page, there is a "Responsible AI dashboard" tab where we can view the insights which we have just uploaded.

In [ ]:
# Remove the temporary directories
from pathlib import Path
import shutil

comp_dir = Path("./Text_classification_component_src")
list_dir = [comp_dir]

for dir in list_dir:
    if dir.exists() and dir.is_dir():
        shutil.rmtree(dir)


list_file = ["./FinacialNewsTrainingComp.yaml"]

for file in list_file:
    if os.path.exists(file):
        os.remove(file)